In [2]:
## Install NeMo
BRANCH = 'v1.8.2'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2
!pip install jiwer

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml --no-check-certificate

  Cloning https://github.com/NVIDIA/NeMo.git (to revision v1.0.0) to c:\users\coco\appdata\local\temp\pip-install-xd0g0tih\nemo-toolkit_e7f9013c8ab448e991ecf62340f54388
  Resolved https://github.com/NVIDIA/NeMo.git to commit 659464487b5b11d7a599f87cb177c5feca3dfe92

  Running command git clone -q https://github.com/NVIDIA/NeMo.git 'C:\Users\coco\AppData\Local\Temp\pip-install-xd0g0tih\nemo-toolkit_e7f9013c8ab448e991ecf62340f54388'
  Running command git checkout -b v1.0.0 --track origin/v1.0.0
  Branch 'v1.0.0' set up to track remote branch 'v1.0.0' from 'origin'.
  Switched to a new branch 'v1.0.0'


In [ ]:
## Importing libraries 
import nemo
import nemo.collections.asr as nemo_asr
import jiwer
from jiwer import wer
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
from omegaconf import DictConfig
import pytorch_lightning as pl

In [5]:
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

[NeMo I 2022-05-14 02:00:18 cloud:56] Found existing object C:\Users\coco\.cache\torch\NeMo\NeMo_1.0.0\QuartzNet15x5Base-En\2b066be39e9294d7100fb176ec817722\QuartzNet15x5Base-En.nemo.
[NeMo I 2022-05-14 02:00:18 cloud:62] Re-using file from: C:\Users\coco\.cache\torch\NeMo\NeMo_1.0.0\QuartzNet15x5Base-En\2b066be39e9294d7100fb176ec817722\QuartzNet15x5Base-En.nemo
[NeMo I 2022-05-14 02:00:18 common:675] Instantiating model from pre-trained checkpoint
[NeMo I 2022-05-14 02:00:19 features:252] PADDING: 16
[NeMo I 2022-05-14 02:00:19 features:269] STFT using torch
[NeMo I 2022-05-14 02:00:19 modelPT:434] Model EncDecCTCModel was successfully restored from C:\Users\coco\.cache\torch\NeMo\NeMo_1.0.0\QuartzNet15x5Base-En\2b066be39e9294d7100fb176ec817722\QuartzNet15x5Base-En.nemo.


In [10]:
# test WER without tuning 
data_dir = 'C:/Users/coco/Desktop/CHANNEL0/WAVE'
script_dir = 'C:/Users/coco/Desktop/CHANNEL0/SCRIPT/'
speakers = ['0010', '0016', '0019', '0039', '0042']
transcript = []
predicted = []
count = 0

for i in speakers: 
  path_unzipped = data_dir + '/SPEAKER' + i
  if not os.path.exists(path_unzipped):
    path = data_dir + '/SPEAKER' + i + '.zip'
    zip_ref = zipfile.ZipFile(path, 'r')
    zip_ref.extractall(path=data_dir)
  n = 0 
  wav_folder = data_dir + '/SPEAKER' + i + '/SESSION'
  while os.path.exists(wav_folder + str(n)):
    files = []
    for filename in os.listdir(wav_folder + str(n)):
      files.append(filename)
    files = sorted(files)
    script_path = script_dir + filename[:6] + '.TXT'
    with open(script_path, encoding = 'utf-8') as script:
      for filename in files:
        script.readline()
        line = script.readline().strip()
        words = line.split()
        edited_words = []
        for word in words:
          if (word != '<SPK/>') and (word != '**'):
            edited_words.append(word.lower())
        transcript_line = ""
        for word in edited_words: 
          transcript_line = transcript_line + word + " "
        transcript_line = transcript_line[:-1]
        transcript.append(transcript_line)
        wav_path = os.path.join(wav_folder + str(n), filename)
        files = [wav_path]
        predicted_line = quartznet.transcribe(paths2audio_files=files)[0]
        predicted.append(predicted_line)
        count = count+1
        #print('"' + transcript_line + '" was predicted as "' + predicted_line + '"')
    n = n+1

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))
print('Number of Testcases (Lines): ' + str(count))

Word Error Rate: 0.08931655563779917
Number of Testcases (Lines): 2373


In [12]:
## Training with more datasets 
train_manifest_path = 'C:/Users/coco/Desktop/CHANNEL0/train_manifest.json'
val_manifest_path = 'C:/Users/coco/Desktop/CHANNEL0/val_manifest.json'
data_dir = 'C:/Users/coco/Desktop/CHANNEL0/WAVE'
script_dir = 'C:/Users/coco/Desktop/CHANNEL0/SCRIPT/'
train_speakers = ['0091', '0124', '0135', '0148', '0173', '0178', '0190', '0194', '0208', '0218', '0245', '0276', '0289', '0308', '0314', '0317', '0329', '0330', '0337', '0349', '0379', '0397', '0432', '0443', '0451', '0462', '0488', '0489', '0490', '0501', '0537', '0543', '0557', '0591', '0592', '0603', '0620', '0627', '0634', '0658', '0659', '0675', '0743', '0762', '0763', '0766', '0778', '0780', '0785', '0790', '0791', '0792', '0798', '0799', '0812', '0831', '0872', '0881', '1058', '1064', '1444']
val_speakers = ['0044', '0047', '0056', '0066', '0074']

## Function to build a manifest
def build_manifest(manifest_path, speakers):
  count = 0
  with open(manifest_path, 'w') as fout: 
    for i in speakers: 
      path_unzipped = data_dir + '/SPEAKER' + i
      if not os.path.exists(path_unzipped):
        path = data_dir + '/SPEAKER' + i + '.zip'
        zip_ref = zipfile.ZipFile(path, 'r')
        zip_ref.extractall(path=data_dir)
      n = 0 
      wav_folder = data_dir + '/SPEAKER' + i + '/SESSION'
      while os.path.exists(wav_folder + str(n)):
        files = []
        for filename in os.listdir(wav_folder + str(n)):
          files.append(filename)
        files = sorted(files)
        script_path = script_dir + filename[:6] + '.TXT'
        with open(script_path, encoding = 'utf-8') as script:
          for filename in files:
            #getting transcript 
            script.readline()
            line = script.readline().strip()
            words = line.split()
            edited_words = []
            for word in words:
              if (word != '<SPK/>') and (word != '**'):
                edited_words.append(word.lower())
            transcript_line = ""
            for word in edited_words: 
              transcript_line = transcript_line + word + " "
            transcript_line = transcript_line[:-1]
            #getting audio file
            audio_path = os.path.join(wav_folder + str(n), filename)
            duration = librosa.core.get_duration(filename=audio_path)
            count = count+1
            # Write the metadata to the manifest
            metadata = {
                "audio_filepath": audio_path,
                "duration": duration,
                "text": transcript_line
            }
            json.dump(metadata, fout)
            fout.write('\n')
        n = n+1
  return count 

train_count = build_manifest(train_manifest_path, train_speakers)
val_count = build_manifest(val_manifest_path, val_speakers)
print("Number of Training Testcases(Lines): " + str(train_count))
print("Number of Validation Testcases(Lines): " + str(val_count))
print("***Done***")

Number of Training Testcases(Lines): 23871
Number of Validation Testcases(Lines): 2373
***Done***


In [18]:
## Config Information 
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML
config_path = './configs/config.yaml'

yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
print(params)

{'name': 'QuartzNet15x5', 'sample_rate': 16000, 'repeat': 1, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'model': {'train_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}, 'validation_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'shuffle': False}, 'preprocessor': {'_target_': 'nemo.collections

In [20]:
## Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

[NeMo I 2022-05-14 03:00:53 cloud:56] Found existing object C:\Users\coco\.cache\torch\NeMo\NeMo_1.0.0\QuartzNet15x5Base-En\2b066be39e9294d7100fb176ec817722\QuartzNet15x5Base-En.nemo.
[NeMo I 2022-05-14 03:00:53 cloud:62] Re-using file from: C:\Users\coco\.cache\torch\NeMo\NeMo_1.0.0\QuartzNet15x5Base-En\2b066be39e9294d7100fb176ec817722\QuartzNet15x5Base-En.nemo
[NeMo I 2022-05-14 03:00:53 common:675] Instantiating model from pre-trained checkpoint
[NeMo I 2022-05-14 03:00:53 features:252] PADDING: 16
[NeMo I 2022-05-14 03:00:53 features:269] STFT using torch


[NeMo W 2022-05-14 03:00:53 nemo_logging:349] C:\Users\coco\anaconda3\lib\site-packages\nemo\collections\asr\parts\features.py:302: FutureWarning: Pass sr=16000, n_fft=512 as keyword args. From version 0.10 passing these as positional arguments will result in an error
      librosa.filters.mel(sample_rate, self.n_fft, n_mels=nfilt, fmin=lowfreq, fmax=highfreq), dtype=torch.float
    


[NeMo I 2022-05-14 03:00:54 modelPT:434] Model EncDecCTCModel was successfully restored from C:\Users\coco\.cache\torch\NeMo\NeMo_1.0.0\QuartzNet15x5Base-En\2b066be39e9294d7100fb176ec817722\QuartzNet15x5Base-En.nemo.


In [21]:
#Freezing the encoder is generally helpful to limit computation and enable faster training; however, in many experiments, freezing the encoder in its entirety will often prevent a model from learning on low-resource languages.
#In order to enable a frozen encoder model to learn on a new language stably, we, therefore, unfreeze the batch normalization layers in the encoder. On top of this, if the model contains "SqueezeExcite" sub-modules, we unfreeze them as well.
def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

quartznet.encoder.freeze()
quartznet.encoder.apply(enable_bn_se)
print("Model encoder has been frozen, and batch normalization has been unfrozen")

Model encoder has been frozen, and batch normalization has been unfrozen


In [22]:
# add in manifest paths 
params['model']['train_ds']['manifest_filepath'] = train_manifest_path
params['model']['validation_ds']['manifest_filepath'] = val_manifest_path

In [24]:
# lr decrease for fine-tuning 
new_opt = copy.deepcopy(params['model']['optim'])
new_opt['lr'] = 0.001

In [32]:
# Use the smaller learning rate we set before
quartznet.setup_optimization(optim_config=DictConfig(new_opt))

# Point to the data we'll use for fine-tuning as the training set
quartznet.setup_training_data(train_data_config=params['model']['train_ds'])

# Point to the new validation data for fine-tuning
quartznet.setup_validation_data(val_data_config=params['model']['validation_ds'])

[NeMo W 2022-05-14 04:58:07 modelPT:640] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2022-05-14 04:58:07 modelPT:748] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )


[NeMo W 2022-05-14 04:58:07 lr_scheduler:600] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2022-05-14 04:58:08 collections:173] Dataset loaded with 23871 files totalling 34.19 hours
[NeMo I 2022-05-14 04:58:08 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2022-05-14 04:58:08 collections:173] Dataset loaded with 2373 files totalling 3.02 hours
[NeMo I 2022-05-14 04:58:08 collections:174] 0 files were filtered totalling 0.00 hours


In [ ]:
#create a PyTorch Lightning trainer and call `fit` again.
trainer = pl.Trainer(max_epochs=2, devices=1, accelerator='gpu')
trainer.fit(quartznet)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
[NeMo W 2022-05-18 02:34:35 modelPT:640] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2022-05-18 02:34:35 modelPT:748] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )


[NeMo W 2022-05-18 02:34:35 lr_scheduler:600] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 29.7 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
107 K     Trainable params
18.8 M    Non-trainable params
18.9 M    Total params
75.698    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
# test new WER after fine tuning 
data_dir = '/content/drive/My Drive/CHANNEL0/WAVE'
script_dir = '/content/drive/My Drive/CHANNEL0/SCRIPT/'
speakers = ['0010', '0016', '0019', '0039', '0042']
transcript = []
predicted = []
count = 0

for i in speakers: 
  path_unzipped = data_dir + '/SPEAKER' + i
  if not os.path.exists(path_unzipped):
    path = data_dir + '/SPEAKER' + i + '.zip'
    zip_ref = zipfile.ZipFile(path, 'r')
    zip_ref.extractall(path=data_dir)
  n = 0 
  wav_folder = data_dir + '/SPEAKER' + i + '/SESSION'
  while os.path.exists(wav_folder + str(n)):
    files = []
    for filename in os.listdir(wav_folder + str(n)):
      files.append(filename)
    files = sorted(files)
    script_path = script_dir + filename[:6] + '.TXT'
    with open(script_path, encoding = 'utf-8') as script:
      for filename in files:
        script.readline()
        line = script.readline().strip()
        words = line.split()
        edited_words = []
        for word in words:
          if (word != '<SPK/>') and (word != '**'):
            edited_words.append(word.lower())
        transcript_line = ""
        for word in edited_words: 
          transcript_line = transcript_line + word + " "
        transcript_line = transcript_line[:-1]
        transcript.append(transcript_line)
        wav_path = os.path.join(wav_folder + str(n), filename)
        files = [wav_path]
        predicted_line = quartznet.transcribe(paths2audio_files=files)[0]
        predicted.append(predicted_line)
        count = count+1
        #print('"' + transcript_line + '" was predicted as "' + predicted_line + '"')
    n = n+1

In [ ]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))
print('Number of Testcases (Lines): ' + str(count))

In [ ]:
#saving the model
torch.save(quartznet.state_dict(), 'model_weights.pth')

#load the model 
#quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")
#quartznet.load_state_dict(torch.load('model_weights.pth'))
#quartznet.eval()